In [1]:
from openai import OpenAI
import json
import re
from datasets import load_dataset
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

key = os.getenv("API_KEY")
client = OpenAI(api_key=key, base_url="https://api.deepseek.com")

In [3]:
dataset = load_dataset("eriktks/conll2003")
ner_dict = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [6]:
def get_ner_label(value, ner_dict):
    for k, v in ner_dict.items():
        if v == value:
            return k
    return None

In [7]:
def send_prompt(prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            #{"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )

    return response.choices[0].message.content

### VANILLA METHOD

In [ ]:
for sentence in dataset['train']['tokens']:
    prompt = f"Given the entity label set: {list(ner_dict.keys())}.\n \
        Based on the given entity label set, please recognize the named entities in the given text.\n \
        Return only a list of tuples with each token and its label, nothing else. For example [('In','O'), ('America','I-LOC'), ('is','O'), ('cold','O'), ...]\n \
        Text: {" ".join(sentence)}"

    answer = send_prompt(prompt)
    # print(answer)
    with open("vanilla.txt", "a") as file:
        file.write(f"{answer}\n")


### OUR METHOD 1

### OUR METHOD 2

### EVALUATION

In [15]:
# Evaluation of results
import ast
from sklearn.metrics import precision_score, recall_score, f1_score

def parse_prediction(pred_str, ner_dict):
    pred_list = ast.literal_eval(pred_str)
    tokens = []
    label_ids = []
    for token, label in pred_list:
        tokens.append(token)
        label_ids.append(ner_dict.get(label, 0))
    return tokens, label_ids
        
def f1Score(file_path, dataset, dict=ner_dict):
    """
    Calculate the F1 score for the predictions in the file.
    """

    label_list = list(set(dict.values()))
    
    with open(file_path, 'r') as f:
        predictions = f.readlines()

    y_pred = []
    y_true = dataset['train']['ner_tags'][:len(predictions)]

    for i, pred in enumerate(predictions):

        pred_tokens, pred_labels = parse_prediction(pred.strip(), ner_dict)
        true_tokens = dataset['train']['tokens'][i]
        true_labels = dataset['train']['ner_tags'][i]

        aligned_preds = []
        pred_idx = 0

        if len(pred_labels) != len(true_labels):
            for true_token in true_tokens:
                if pred_idx < len(pred_tokens) and pred_tokens[pred_idx] == true_token:
                    aligned_preds.append(pred_labels[pred_idx])
                    pred_idx += 1
                else:
                    aligned_preds.append(-1)
        else:
            aligned_preds = pred_labels[:]
    
        y_pred.append(aligned_preds)


    flat_true = [label for seq in y_true for label in seq]
    flat_pred = [label for seq in y_pred for label in seq]
    
    precision = precision_score(flat_true, flat_pred, average='micro', zero_division=0)
    recall = recall_score(flat_true, flat_pred, average='micro', zero_division=0)
    f1 = f1_score(flat_true, flat_pred, average='micro', zero_division=0)

    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    return f1

In [16]:
f1Score("vanilla.txt", dataset, ner_dict)

Precision: 0.9291
Recall:    0.9291
F1 Score:  0.9291


0.9290897781463819